# [George McNinch](http://gmcninch.math.tufts.edu) Math 87 - Spring 2025

# Week 15

# Racket and linear programming 

# Racket and GLPK 

The main point of this notebook is to convince you that other tools than `python` and `numpy` are available for some of the applications we've used in the course this term.

I'm going to show how to use the `racket scheme` language to solve some linear programs.

In fact, the `racket` [library for solving linear programs](https://docs.racket-lang.org/glpk/index.html#%28part._.The_.Linear_.Programming_problem%29) 
depends on the [GNU Linear Programming Kit](https://www.gnu.org/software/glpk/) (which describes itself as

>  a set of routines written in ANSI C and organized in the form of a callable library.
)

To use it, I had to do a bunch of stuff:
- install the `GNU GPLK` (`sudo apt install glpk-utils`)
- (install `racket` (see https://racket-lang.org/download/)
- install the corresponding `racket` package (`raco pkg install GLPK`)

In [1]:
#lang iracket/lang #:require racket

(require glpk)

Here is the technical specification for the solver:

```
(lp-solve	    objective	 
     	 	    direction	 
     	 	    constraints	 
     	 	    bounds	 
     	 	    #:terminal-output terminal-output?)	 
→ (list/c symbol?
          (or/c symbol? false?)
          (list/c flonum? (listof (list/c symbol? flonum?))))

  objective : objective?
  direction : (or/c 'max 'min)
  constraints : (listof constraint?)
  bounds : (listof bound?)
  terminal-output? : boolean?
```

The objective function and constraints are described as follows:

> Both the objective and the constraints make use of a "linear combination" form:
>
>>    lin-comb? = (listof (list/c real? symbol?))
>
> ... representing a linear combination of structural variables.
>
> The objective function includes a constant term and a linear combination of structural variables:
>
>>    objective? = (pair/c real? lin-comb?)
>
> The constraints each include the name of an auxiliary variable and a linear combination of structural variables:
>
>>    constraint? = (pair/c symbol? lin-comb?)

In `racket`, a `lin-comb?` would therefore be something like

> `'((1 a) (2 b) (3 c))`

You also have to provide bounds:

> You must provide bounds for every auxiliary and structural variable.

>>    bound?    = (list/c symbol? lo-bound? hi-bound?)  
>>    lo-bound? = (or/c 'neginf real?)  
>>    hi-bound? = (or/c 'posinf real?)

e.g. our bounds might be something like

> `'((a 0 posinf) (b 0 posinf) (c neginf 0))`

> The result is a list containing a symbol that indicates either success `('good)` or one of two kinds of failure, and then a symbol which in case of failure conveys information  about the nature of the failure, and then either a solution or false. 

> When the first symbol in the result is `'bad-result`, the second element is a `FailCode` (definition below). When the first symbol in the result is `'bad-status`, the second element is a `SolutionStatus` (also defined below).

# Example

I'm going to repeat the example from [the racket docs](https://docs.racket-lang.org/glpk/index.html#%28part._.The_.Linear_.Programming_problem%29).

> Okay, let’s say you’re trying to figure out whether you have enough food for your picnic. In particular, you’re buying hamburgers, slices of bread, and pickles. You have three kinds of guests: Children, Adults, and Chickens.

> Each adult wants one slice of bread, a patty, and two pickles. Each child wants two slices of bread, and a patty. Each chicken wants *either* one slice of bread, one patty, or one pickle.

> So, if we have 30 slices of bread, 20 patties, and 50 pickles, how many guests can we invite?

> Let’s make up some variables. We’ll use `k` for the number of children, `a` for the number of adults, `cb` for the chickens eating a slice of bread, `cp` for the chickens eating a patty, and `ck` for the chickens eating a pickle. For our auxiliary variables, we’ll use `fb` for the number of slices of bread eaten, `fp` for the number of patties eaten, and `fk` for the number of pickles eaten.

So we have

> `f_b = a + 2k + c_b`  
> `f_p = a + k + c_p`  
> `f_k = 2a + c_k`  

For bounds we use:

> `0 < f_b < 30`  
> `0 < f_p < 20`  
> `0 < f_k < 50`  
> `0 < a, k, c_p, c_b, c_k`

Finally, we aim to maximize the *objective function*

> `g = a + k + c_p + c_b + c_k`

We use the following:

In [2]:
(lp-solve
 '(0 (1 a) (1 k) (1 cb) (1 cp) (1 ck))  ;; this is the objective function
                                        ;; with constant term 0.
 'max                                   ;; direction arg
 '((fb (1 a) (2 k) (1 cb))              ;; constraint arg
   (fp (1 a) (1 k) (1 cp))
   (fk (2 a) (1 ck)))
 '((fb 0 30)                            ;; bounds arg
   (fp 0 20)
   (fk 0 50)
   (a 0 posinf)
   (k 0 posinf)
   (cb 0 posinf)
   (cp 0 posinf)
   (ck 0 posinf)))

'(good #f (100.0 ((a 0.0) (k 0.0) (cb 30.0) (cp 20.0) (ck 50.0))))

In [3]:
(lp-solve
 '(0 (8 a) (10 k) (1/2 cb) (1/2 cp) (1/2 ck))  ;; this is the objective function
                                        ;; with constant term 0.
 'max                                   ;; direction arg
 '((fb (1 a) (2 k) (1 cb))              ;; constraint arg
   (fp (1 a) (1 k) (1 cp))
   (fk (2 a) (1 ck)))
 '((fb 0 30)                            ;; bounds arg
   (fp 0 20)
   (fk 0 50)
   (a 0 posinf)
   (k 0 posinf)
   (cb 0 posinf)
   (cp 0 posinf)
   (ck 0 posinf)))

'(good #f (195.0 ((a 10.0) (k 10.0) (cb 0.0) (cp 0.0) (ck 30.0))))

Note that the interface to the `lp-solve` function is somewhat different than the interface to `numpy`'s `linprog` function.

In particular, here we are allowed to define auxiliary variables, and use them in the bounds. The functions are *equivalent* but they feel a bit different.

> We can add arbitrary further constraints on this: each chicken must be chaperoned by an adult, each chicken must be chaperoned by an adult, no adult can chaperone both a child and a chicken.

> To model this, we divide adults into adults chaperoning kinds (ak) and adults chaperoning chickens (ac). We could replace a entirely, but it’s easier just to require that a is the sum of ac and ak. Also, let’s bump up the desirability of chickens, just to get a more interesting result:

In [4]:
(lp-solve
 '(0 (8 a) (10 k) (2 cb) (2 cp) (2 ck))
 'max
 '((fb (1 a) (2 k) (1 cb))
   (fp (1 a) (1 k) (1 cp))
   (fk (2 a) (1 ck))
   (z (-1 a) (1 ak) (1 ac))
   (excessak (1 ak) (-1 k))
   (excessac (1 ac) (-1 cb) (-1 cp) (-1 ck)))
 '((z 0 0)
   (ak 0 posinf)
   (ac 0 posinf)
   (excessak 0 posinf)
   (excessac 0 posinf)
   (fb 0 30)
   (fp 0 20)
   (fk 0 50)
   (a 0 posinf)
   (k 0 posinf)
   (cb 0 posinf)
   (cp 0 posinf)
   (ck 0 posinf)))

'(good #f (200.0 ((a 20.0) (k 0.0) (cb 10.000000000000002) (cp 0.0) (ck 9.999999999999998) (ak 0.0) (ac 20.0))))

Notice that `ak` and `ac` appear in the bounds, but not in as auxiliary variables via the constraints. So they are treated as "decision variables" and reported in the output of `lp-sovle`.

# Integer programming!

There is a (mixed) integer programming solver, as well!

```
(mip-solve  objective	 
     	 	direction	 
     	 	constraints	 
     	 	bounds	 
     	 	integer-vars	 
     	 	#:terminal-output terminal-output?	 
     	 	#:time-limit time-limit)	 

→  (or/c (list/c symbol?
         (or/c symbol? #f)
         (or/c (list/c flonum? (listof (list/c symbol? flonum?)))
                #f)))

  objective : objective?
  direction : (or/c 'max 'min)
  constraints : (listof constraint?)
  bounds : (listof bound?)
  integer-vars : (listof symbol?)
  terminal-output? : boolean?
  time-limit : (or/c false? natural?)
```

> The Mixed-Integer-Programming solver is an extension of the linear programming solver, and the problems that it solves are an extension of linear programming problems. Specifically, in a mixed integer programming problem, some of the solution variables can be labeled as integer variables, whose values must be integers.

> Mixed integer programming is ... well, a lot harder than simple linear programming. In fact, the first step in MIP is to solve the corresponding linear programming problem, where the variables are all allowed to take on non-integer variables. The branch-and-cut algorithm then attempts to find a related solution where the specified structural variables have integer values.

> This problem is NP-hard, so ... it can take a while.

# MIP example

> Here’s a simple example: we need at least 4.5 sections of csc101 and 202. We have two instructors, smith and martinez, each of whom can teach 9 sections. Can we staff both of our classes? Yes.

In [6]:
(mip-solve '(0 (1 smith-extra) (1 martinez-extra)) 
           'max
           '((csc101-offered (1 smith-csc101) (1 martinez-csc101))
             (csc202-offered (1 smith-csc202) (1 martinez-csc202))
             (smith-secns (1 smith-csc101)
                          (1 smith-csc202)
                          (1 smith-extra))
             (martinez-secns (1 martinez-csc101)
                             (1 martinez-csc202)
                             (1 martinez-extra)))
           '((csc101-offered 4.5 posinf)
             (csc202-offered 4.5 posinf)
             (smith-secns 9 9)
             (martinez-secns 9 9)
             (smith-csc101 0 posinf)
             (smith-csc202 0 posinf)
             (smith-extra 0 posinf)
             (martinez-csc101 0 posinf)
             (martinez-csc202 0 posinf)
             (martinez-extra 0 posinf))
           '()
           ;;'(smith-csc101 smith-csc202                      ;; integer-vars argument
           ;;               martinez-csc101 martinez-csc202)
           )

'(good #f (9.0 ((smith-csc101 4.5) (martinez-csc101 0.0) (smith-csc202 4.5) (martinez-csc202 0.0) (smith-extra 0.0) (martinez-extra 9.0))))